#### Things that I've accomplished so far: 
1) Worked through the paper to extent that I needed to
2) Working prototype of obtaining the forward process

#### Next step
1) Fundamental working prototype of obtaining the loss for the reverse diffusion kernel

In [1]:
## imports, etc

import jax
import jax.numpy as jnp
from jax import random

import matplotlib.pyplot as plt

# Let's just try something, why not? 
# I'm smart and quick on my feet, there's no need to be scared
import tensorflow as tf  # Used for the data loading pipeline
import tensorflow_datasets as tfds

# I'll simplify this at some point, as it's pretty annoying, but it's
# not super important right now
def get_mnist_dataset(batch_size=64):
    """
    Downloads and prepares the MNIST dataset using tensorflow_datasets,
    and returns an iterator that yields JAX-compatible NumPy arrays.
    
    Returns:
        An iterator that yields batches of (images, labels).
    """
    
    # --- 1. Load the Dataset with TFDS ---
    # as_supervised=True returns (image, label) tuples.
    ds = tfds.load('mnist', split='train', as_supervised=True)
    
    # --- 2. Define the Preprocessing Function ---
    # This function will be mapped over the dataset.
    # TFDS loads images as uint8 [0, 255].
    def normalize_img(image, label):
        # Cast to float
        image = tf.cast(image, tf.float32)
        # Normalize to [-1, 1]
        # (pixel / 127.5) - 1.0
        # (0 / 127.5) - 1.0 = -1.0
        # (255 / 127.5) - 1.0 = 2.0 - 1.0 = 1.0
        image = (image / 127.5) - 1.0
        return image, label

    # --- 3. Build the tf.data Pipeline ---
    ds = ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.cache()  # Cache the preprocessed data in memory
    ds = ds.shuffle(1000)  # Shuffle the dataset
    ds = ds.batch(batch_size)  # Batch the data
    ds = ds.prefetch(tf.data.AUTOTUNE)  # Prefetch batches for performance
    
    # --- 4. Convert to NumPy Iterator ---
    # tfds.as_numpy converts the tf.Tensor batches into NumPy arrays,
    # which JAX can consume without any issues.
    # ~ I am guessing where this step differs between JAX and Pytorch
    return tfds.as_numpy(ds)

# --- Example Usage ---
if __name__ == "__main__":
    
    # To run this, you'll need: 
    # pip install jax jaxlib tensorflow tensorflow-datasets
    
    # JAX models are pure functions, so the data pipeline is
    # typically an iterator that yields NumPy arrays.
    
    print("Loading MNIST data with JAX / tensorflow_datasets...")
    train_iter = get_mnist_dataset(batch_size=128)
    
    # Get one batch of data
    # 'images_np' will be a NumPy array of shape [batch_size, 28, 28, 1]
    # 'labels_np' will be a NumPy array of shape [batch_size]
    images_np, labels_np = next(iter(train_iter))
    
    # You can then convert them to JAX arrays for use in a train step
    images_jnp = jnp.array(images_np)
    labels_jnp = jnp.array(labels_np)
    
    print(f"\n--- JAX Data Info ---")
    print(f"Data iterator created successfully.")
    print(f"Batch of images shape: {images_jnp.shape}")
    print(f"Data format: [Batch Size, Height, Width, Channels] (Channel-Last)")
    print(f"Labels shape: {labels_jnp.shape}")
    print(f"Image data type: {images_jnp.dtype}")
    print(f"Image min value: {images_jnp.min():.2f}")
    print(f"Image max value: {images_jnp.max():.2f}")
    print(f"Label data type: {labels_jnp.dtype}")

2025-11-17 10:50:04.807462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763405404.851891    5110 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763405404.864662    5110 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763405404.971006    5110 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763405404.971026    5110 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763405404.971029    5110 computation_placer.cc:177] computation placer alr

Loading MNIST data with JAX / tensorflow_datasets...


2025-11-17 10:50:08.057170: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-11-17 10:50:08.213749: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-11-17 10:50:08.213882: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take


--- JAX Data Info ---
Data iterator created successfully.
Batch of images shape: (128, 28, 28, 1)
Data format: [Batch Size, Height, Width, Channels] (Channel-Last)
Labels shape: (128,)
Image data type: float32
Image min value: -1.00
Image max value: 1.00
Label data type: int32


Idea: 
1) Let's take one image, and completely transform it to white Gaussian noise
2) Try to compute the sigma and mu estimator for just one step---does this require having an MLP, CNN, etc.?

Looks like we'll need an MLP in order to actually implement the estimator, and therefore get a score off of how it will learn

Yes, indeed, they implement an MLP architecture which has hidden layers and convolutional layers. So, what I need to do is first figure out how to actually build conv layers in JAX, then I'll need to figure out how to build the MLP in JAX. Then, I'll circle back to actually implementing the version that's in the paper.